# DataCleaning Notebook - AAAG

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import numpy as np
import re
pd.set_option('display.max_columns', None)

In [2]:
df_train = pd.read_csv('train.csv',index_col ="Id")
df_train.shape

(1460, 80)

In [3]:
df_train.SalePrice = np.log1p(df_train.SalePrice)
df_train.sample(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
483,70,RM,50.0,2500,Pave,Pave,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,7,8,1915,2005,Gable,CompShg,Stucco,Stucco,None,0.0,Gd,TA,PConc,TA,TA,No,ALQ,299,Unf,0,611,910,GasA,Ex,Y,SBrkr,916,910,0,1826,1,0,1,1,4,1,Ex,7,Min2,1,Gd,Attchd,1915.0,Unf,1,164,Fa,Fa,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal,11.951187
1336,20,RL,80.0,9650,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,5,1977,1977,Gable,CompShg,Plywood,Plywood,BrkFace,360.0,TA,TA,CBlock,Gd,TA,No,ALQ,686,Unf,0,664,1350,GasA,TA,Y,SBrkr,1334,0,0,1334,0,1,2,0,2,1,TA,6,Typ,1,TA,Attchd,1977.0,RFn,2,630,TA,TA,Y,0,16,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal,12.031130
900,20,RL,65.0,6993,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Feedr,Norm,1Fam,1Story,5,7,1961,1994,Gable,CompShg,HdBoard,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,465,Unf,0,447,912,GasA,TA,Y,SBrkr,1236,0,0,1236,0,0,1,0,3,1,TA,6,Typ,1,TA,Attchd,1961.0,Unf,1,288,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,11.813037
610,20,RL,61.0,7943,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Feedr,Norm,1Fam,1Story,4,5,1961,1961,Gable,CompShg,VinylSd,VinylSd,BrkCmn,192.0,TA,Fa,CBlock,TA,TA,Mn,Rec,903,Unf,0,126,1029,GasA,Gd,Y,SBrkr,1029,0,0,1029,1,0,1,0,3,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1,261,TA,TA,Y,64,0,39,0,0,0,NaN,NaN,NaN,0,4,2007,WD,Normal,11.682677
1035,30,RL,50.0,6305,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,5,7,1938,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,PConc,Fa,Fa,No,Unf,0,Unf,0,920,920,GasA,Ex,Y,SBrkr,954,0,0,954,0,0,1,0,2,1,Fa,5,Typ,1,Gd,Basment,1938.0,Unf,1,240,Fa,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,11.693170
397,20,RL,60.0,7200,Pave,NaN,Reg,Low,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,5,5,1972,1972,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Av,Rec,777,Unf,0,117,894,GasA,TA,Y,SBrkr,894,0,0,894,0,0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1985.0,RFn,2,600,TA,TA,Y,215,0,0,0,0,0,NaN,NaN,NaN,0,9,2009,WD,Normal,11.719948
1324,30,RL,50.0,5330,Pave,NaN,Reg,HLS,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1Story,4,7,1940,1950,Hip,CompShg,VinylSd,VinylSd,None,0.0,Fa,TA,CBlock,TA,TA,No,LwQ,280,Unf,0,140,420,GasA,Gd,Y,SBrkr,708,0,0,708,0,0,1,0,2,1,Fa,5,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,164,0,0,0,0,0,NaN,NaN,NaN,0,12,2009,WD,Normal,11.320566
1088,60,FV,85.0,10574,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,8,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Mn,Unf,0,Unf,0,1082,1082,GasA,Ex,Y,SBrkr,1082,871,0,1953,0,0,2,1,3,1,Gd,9,Typ,1,Gd,Attchd,2005.0,RFn,3,1043,TA,TA,Y,160,50,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,12.437188
602,50,RM,50.0,9000,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1.5Fin,6,6,1937,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,Gd,PConc,TA,TA,No,Unf,0,Unf,0,780,780,GasA,TA,Y,SBrkr,780,595,0,1375,0,0,1,1,3,1,Gd,6,Typ,1,Gd,Detchd,1979.0,Unf,1,544,TA,TA,P,0,162,0,0,126,0,NaN,NaN,NaN,0,12,2007,WD,Normal,11.856522


In [4]:
#define function to dummify continuous porch, deck and pool variables
def make_binary(X,vrs):
    '''
    This is a mutating function that replaces any nonzero value with 1
    ----------
    X: dataframe
    vrs: list of variables or single variable
    '''
    for var in vrs:
        df_train.loc[df_train[var]>0,var]=1

In [5]:
def my_impute(df_col,method='replace',replace_with=None,missing_string=None,convert_to_numeric='infer'):
    '''
    - This function takes works for the columns mentioned above. 
    - df_col: take a column series 
    - replace: replace a null with a string defined by replace_with
    - replace_with: the string that replaces the null value
    - missing_string: define what constitutes a null value which will be replaced
    - convert_to_numeric: takes in "infer", "force", "no". "Infer" automatically senses what is 
    - the best action. Converts variable type to numeric. 
    '''
    new_series=df_col.copy()
    try:
        if (df_col.str.contains('Shed').any() and df_col.str.contains('TenC').any()):
            misc_dict={'Gar2':'Shed',
                      'Othr':'No Shed',
                      'TenC':'No Shed'}
            new_series.replace(misc_dict,inplace=True)
        if missing_string!=None:
            missing_dict={null_word:None for i in missing_string}
            new_series.replace(missing_dict,inplace=True) 
        if method=='replace':
            if replace_with==None:
                raise ValueError('Must provide replacement string!')
            new_series[new_series.isnull()]=replace_with
        elif method=='mode':
            new_series[new_series.isnull()]=new_series.mode(dropna=True)[0]
        if convert_to_numeric=='force': 
            score_dict={'TA':3,
                       'Gd':4,
                       'Ex':5,
                       'Fa':2,
                       'Po':1,
                       replace_with:0}
            new_series.replace(score_dict,inplace=True)
            return new_series
        elif convert_to_numeric=='infer':
            if new_series.str.contains('(^Gd$)',regex=True).any() and \
                new_series.str.contains('(^TA$)',regex=True).any() and \
                new_series.str.contains('(^Fa$)',regex=True).any():
                score_dict={'TA':3,
                           'Gd':4,
                           'Ex':5,
                           'Fa':2,
                           'Po':1,
                           replace_with:0}
                new_series.replace(score_dict,inplace=True)
                return new_series
            elif new_series.str.contains('(^Gd$)',regex=True).any() and \
                new_series.str.contains('(^Av$)',regex=True).any() and \
                new_series.str.contains('(^Mn$)',regex=True).any() and \
                new_series.str.contains('(^No$)',regex=True).any():
                score_dict={'Gd':4,
                           'Av':3,
                           'Mn':2,
                           'No':1,
                           replace_with:0}
                new_series.replace(score_dict,inplace=True)
                return new_series
            else:
                return new_series
        elif convert_to_numeric=='no':
            return new_series
        else:
            return new_series
    except:
        try: 
            if method=='mean':
                new_series[new_series.isnull()]=new_series.mean(skipna=True)
                return new_series
            elif method=='median':
                new_series[new_series.isnull()]=new_series.median(skipna=True)[0]
                return new_series
        except Exception as e:
            print(type(e),e)

In [6]:
# Clean:          
# - PavedDrive
# - SaleCondition
# - SaleType
# - Street
# - Central Air
# - Functional
# - Heating
# - 1stFlrSF
# - 2ndFlrSF
# - GrLivArea
# - LotArea
# - MSZone
# - OverallCond
# - OverallQual
# - TotalBsmtSF
# - YearBuilt
# - YearRemodAdd

# Drop
# - PoolQC
# - BsmtFinType2
# - Utilities
# - BsmtFinSF1  
# - BsmtFinSF2  
# - BsmtUnSF 
# - LowQualFinSF
# - KitchenAbvGr 
# - MiscVal variables
df_train = df_train.drop(['PoolQC','TotalBsmtSF','BsmtFinType2','Utilities',\
                          'BsmtFinSF1', 'BsmtFinSF2',\
                          'BsmtUnfSF','LowQualFinSF','KitchenAbvGr', 'MiscVal'], axis=1)

# pply the make binary function to the porch, deck, and pool variables (MUTATING!)
vrs = ['3SsnPorch', 'EnclosedPorch', 'OpenPorchSF','PoolArea','ScreenPorch', 'WoodDeckSF']
make_binary(df_train,vrs)

# Converting to category/string
# - MSSubClass
# - MoSold
df_train['MSSubClass'] = df_train['MSSubClass'].astype('category')
df_train['MoSold'] = df_train['MoSold'].astype('category')

# Impute with mode/median
# - Electrical 
df_train.loc[:,'Electrical']=my_impute(df_train.loc[:,'Electrical'],method='mode',replace_with=None,missing_string=None,convert_to_numeric='no')

# Impute with mean 
# - GarageYrBlt
df_train.loc[:,'GarageYrBlt']=my_impute(df_train.loc[:,'GarageYrBlt'],method='mean',replace_with=None,missing_string=None,convert_to_numeric='no')

# Impute with groupedby median of neighborhood
# - LotFrontage
df_train['LotFrontage'] = df_train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Replace with N/A with “no fence”, "No Alley" "No Basement" etc.
# - Fence
# - Alley
# - GarageFinish
# - GarageType
# - BsmtFinType1
df_train.loc[:,'Fence']=my_impute(df_train.loc[:,'Fence'],method='replace',replace_with='No Fence',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'Alley']=my_impute(df_train.loc[:,'Alley'],method='replace',replace_with='No Alley',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageFinish']=my_impute(df_train.loc[:,'GarageFinish'],method='replace',replace_with='No Garage',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageType']=my_impute(df_train.loc[:,'GarageType'],method='replace',replace_with='No Garage',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtFinType1']=my_impute(df_train.loc[:,'BsmtFinType1'],method='replace',replace_with='No Basement',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'MasVnrType']=my_impute(df_train.loc[:,'MasVnrType'],method='replace',replace_with='No Masonry Veneer',missing_string=None,convert_to_numeric='no')

# Replace with N/A with 0 and turning ordinal into continuous variable
# - BsmtCond
# - BsmtExposure
# - BsmtQual
# - GarageCond
# - GarageQual
# - FireplaceQu
# - HeatingQC
# - KitchenQual
df_train.loc[:,'BsmtCond']=my_impute(df_train.loc[:,'BsmtCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtExposure']=my_impute(df_train.loc[:,'BsmtExposure'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtQual']=my_impute(df_train.loc[:,'BsmtQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageCond']=my_impute(df_train.loc[:,'GarageCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageQual']=my_impute(df_train.loc[:,'GarageQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'FireplaceQu']=my_impute(df_train.loc[:,'FireplaceQu'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'HeatingQC']=my_impute(df_train.loc[:,'HeatingQC'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'KitchenQual']=my_impute(df_train.loc[:,'KitchenQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'ExterCond']=my_impute(df_train.loc[:,'ExterCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'ExterQual']=my_impute(df_train.loc[:,'ExterQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')

# Replace with N/A with 0 only
# - MasVnrArea
# - GarageArea
# - GarageCars
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(0)
df_train['GarageArea'] = df_train['GarageArea'].fillna(0)
df_train['GarageCars'] = df_train['GarageCars'].fillna(0)

# Lumping features into shed and no shed
# - MiscFeature
df_train.loc[:,'MiscFeature']=my_impute(df_train.loc[:,'MiscFeature'],method='replace',replace_with='No Shed',missing_string=None,convert_to_numeric='infer')

# Check: 
df_train.isnull().sum().sum()

/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0

In [7]:
df_train.shape

(1460, 70)

In [8]:
df_train.to_csv('Version_5_df.csv')